In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
for dirname, _, filenames in os.walk('/kaggle/input/proximo-hit-spotify'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Hipoteses

## primeira hipotese: duração da musica afeta o target

# Carregamento dos dados

In [ ]:
pd.read_csv('/kaggle/input/proximo-hit-spotify/sample_submission.csv')

In [3]:
treino = pd.read_csv('/kaggle/input/proximo-hit-spotify/train.csv')
teste = pd.read_csv('/kaggle/input/proximo-hit-spotify/test.csv')

# Tratamento de valores nulos

In [ ]:
# verificando a existência e volume de valores nulos em cada coluna
teste.isnull().sum()

In [5]:
# baixo volume de valores nulos, então podemos lidar com eles dropando as linhas em que eles se encontram
teste = teste.dropna()

In [ ]:
# garantindo que todos os valores nulos foram excluidos do dataset
treino.isnull().sum()

# Processamento dos dados

In [ ]:
# verificando o tipo dos dados
treino.dtypes

## Excluindo variáveis catergóricas que não serão úteis para a predição

In [8]:
# definindo quais colunas vão ser excluídas
# o motivo de excluir essas colunas é que são variáveis irrelevantes ou variáveis com muitos valores categóricos únicos, o que torna dificil de encode 
colunas_para_excluir = ['track_id','artists','album_name','track_name']

In [9]:
# dropando as colunas selecionadas
treino_v2 = treino.drop(columns = colunas_para_excluir)
teste_v2 = teste.drop(columns = colunas_para_excluir)

## Transformar valores 'bool' em 0 ou 1, ao invés de 'True' ou 'False'

In [10]:
# transformando os valores booleanos em int para melhorar a visualização
treino_v2["explicit"] = treino_v2["explicit"].astype(int)
teste_v2["explicit"] = teste_v2["explicit"].astype(int)

In [ ]:
# target encoding na variável de genero musical
# foi escolhido aqui o método target encoding por terem muitos valores únicos, porém ser uma feature de alta importância para a predição 
mean_target_per_genre = treino_v2.groupby('track_genre')['popularity_target'].mean()

treino_v2['genre_encoded'] = treino_v2['track_genre'].map(mean_target_per_genre)
teste_v2['genre_encoded'] = teste_v2['track_genre'].map(mean_target_per_genre)

mean_target_general = treino_v2['popularity_target'].mean()
teste_v2['genre_encoded'] = teste_v2['genre_encoded'].fillna(mean_target_general)

treino_v2

In [12]:
# dropando a coluna original de genero
treino_v3 = treino_v2.drop(columns = ['track_genre'])
teste_v3 = teste_v2.drop(columns = ['track_genre'])

In [ ]:
teste_v3

In [ ]:
treino_v3

# Treinamento do modelo

## Validação cruzada

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# dividindo o dataset em X e Y 
X = treino_v3.drop(columns=['popularity_target'])
y = treino_v3['popularity_target']

# utilizando RandomForest, que costuma ser bom na maioria dos problemas
rf_classifier = RandomForestClassifier()

# fazendo validação cruzada (usando apenas 5 folds para economizar tempo e processamento)
cv_scores = cross_val_score(rf_classifier, X, y, cv=5)

# printando a acurácia do modelo 
print("Cross-validation scores: ", cv_scores)
print("Mean cross-validation score: ", cv_scores.mean())


## Utilizando RandomSearch para achar hiperparametros melhores, possivelmente melhorando a performance do modelo. A escolha do algoritmo é dada pelo fato de que um resultado satisfatorio já foi encontrado, portanto não é necessario utilizar poder computacional e tempo excessivo aplicando um grid search.

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score

# dividindo o dataset em conjunto de treino e validação (80% treino, 20% validação)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# definindo o espaço de busca para os hiperparâmetros
param_distributions = {
    'n_estimators': np.arange(10, 201, 10),  
    'max_depth': np.arange(5, 51, 5),  
    'min_samples_split': np.arange(2, 11),  
    'min_samples_leaf': np.arange(1, 11)  
}

# inicializando o RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=111)

# configurando o RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf_classifier, 
    param_distributions=param_distributions, 
    n_iter=50,  
    cv=3,  
    verbose=1, 
    n_jobs=-1,  
    random_state=111
)

# executando o RandomizedSearchCV
random_search.fit(X_train, y_train)

# fazendo previsões no conjunto de validação
y_pred = random_search.best_estimator_.predict(X_val)

# avaliando a acurácia no conjunto de validação
accuracy = accuracy_score(y_val, y_pred)

# exibindo os melhores hiperparâmetros e a acurácia correspondente
print("Melhores hiperparâmetros: ", random_search.best_params_)
print("Acurácia no conjunto de validação: ", accuracy)

## O finetuning de hiperparâmetros não surtiu tanto efeito nos resultados.

# Teste do modelo

In [ ]:
# dividindo o dataset treino_v3 em X e y
X_train = treino_v3.drop(columns=['popularity_target', 'track_unique_id'])
y_train = treino_v3['popularity_target']

# o dataset teste_v3 só contém o track_unique_id e as features
X_test = teste_v3.drop(columns=['track_unique_id'])  # Certifique-se de que a coluna 'track_unique_id' seja removida

# inicializando o RandomForestClassifier com os melhores hiperparâmetros encontrados
best_params = random_search.best_params_  # Usando os melhores parâmetros encontrados previamente
rf_classifier = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)

# treinando o modelo com o treino_v3
rf_classifier.fit(X_train, y_train)

# fazendo as predições no conjunto de teste_v3
predictions = rf_classifier.predict(X_test)

# criando o dataframe de submissão
submission = pd.DataFrame({
    'track_unique_id': teste_v3['track_unique_id'],  # pegando o track_unique_id do dataset de teste
    'popularity_target': predictions  # previsões feitas pelo modelo
})

# salvando o arquivo no formato CSV para submissão no Kaggle
submission.to_csv('kaggle_submission.csv', index=False)

print("Arquivo de submissão 'kaggle_submission.csv' criado com sucesso!")
